In [ ]:
import requests
import json
url = ('https://newsapi.org/v2/everything?'
       #'category=general&'
       'domains=bbc.co.uk&'
       #'country=gb&'
       'pageSize=99&'
       'page=1&'
       'from=2019-05-05&to=2019-06-03&'
       'apiKey=34e73062364d41b180a6f6a9625bff79')
response = requests.get(url)

with open('data/data.json', 'w') as outfile:
    json.dump(response.json(), outfile)

In [ ]:
import pandas as pd

news_df = pd.DataFrame(columns=['Title', 'Date', 'Stems'])

In [ ]:

with open("data/data.json") as f:
        loaded_json = json.load(f)
        for i in range(len(loaded_json["articles"])):
            if loaded_json["articles"][i]["content"] is not None:
                news_df = news_df.append({'Title': loaded_json["articles"][i]["title"], 'Date': loaded_json["articles"][i]["publishedAt"]}, ignore_index=True)
                #print(loaded_json["articles"][i]["title"])
                

In [ ]:
news_df

## Stemming

In [ ]:
# import spacy
# from spacy import displacy
# from collections import Counter
# import en_core_web_sm
# nlp = en_core_web_sm.load()

In [ ]:
import nltk
from nltk.stem import PorterStemmer
#from nltk.stem import LancasterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')
porter = PorterStemmer()

from nltk.tokenize import sent_tokenize, word_tokenize
def stemSentence(sentence):
    token_words=tokenizer.tokenize(sentence)
    token_words
    stop_words = set(stopwords.words('english'))
    #print(stop_words)
    filter_sentense = [w for w in token_words if not w in stop_words]
    
    stem_sentence=[]
    for word in filter_sentense:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [ ]:
sentence=news_df.Title[0]
x=stemSentence(sentence)
print(sentence)
print(x)


In [ ]:
for i in news_df.index:
    news_df.Stems[i] = stemSentence(news_df.Title[i])

In [ ]:
wordfreq = []
master_wordlist = []
for i in news_df.index:
    wordlist = news_df.Stems[i].split()

    master_wordlist+= wordlist
#print(master_wordlist)
# print(set(master_wordlist))
# for w in master_wordlist:
#     wordfreq.append(master_wordlist.count(w))

# ww = list(zip(master_wordlist, wordfreq))
# print("Pairs\n" + str(ww[:]))

In [ ]:
def wordListToFreqDict(wordlist):
    wordfreq = [wordlist.count(p) for p in wordlist]
    return dict(zip(wordlist,wordfreq))

In [ ]:
word_freq = wordListToFreqDict(master_wordlist)
# print(word_freq)

In [ ]:
# for x, y in word_freq.items():
#     print(x, y)
# print(word_freq.values())

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.io import output_notebook

output_notebook()

In [ ]:
fruits = list(word_freq.keys())
counts = list(word_freq.values())

# sorting the bars means sorting the range factors
sorted_fruits = sorted(fruits, key=lambda x: counts[fruits.index(x)])

p = figure(x_range=sorted_fruits, plot_height=500, plot_width=1400, title="Word frequencies",
           toolbar_location="below")

p.vbar(x=fruits, top=counts, width=0.9)
p.xaxis.major_label_orientation = "vertical"
p.xgrid.grid_line_color = None
p.xaxis.axis_label_text_font_size = "25pt"
p.xaxis.major_label_text_font_size = "10pt"
p.y_range.start = 0

show(p)

## Google trends (hourly data)

In [ ]:
import pandas as pd
from pytrends.request import TrendReq
import os  
import numpy as np
pytrend = TrendReq(hl='en-US', tz=360)

In [ ]:
#functions
def get_interest_and_region(kw_list,pytrend):
    if not (os.path.isfile('Data/'+str(kw_list[0])+'.pkl') and os.path.isfile('Data/'+str(kw_list[0])+'_regions'+'.pkl')):
        print('Term not downloaded yet, downloading now..')
        
        #pytrend = TrendReq(hl='en-US', tz=360, timeout=(10,25), proxies=['https://34.203.233.13:80',], retries=5, backoff_factor=1)

        search_df = pytrend.get_historical_interest(kw_list, year_start=2005, year_end=2018).reset_index()

        search_df.to_pickle('Data/'+str(kw_list[0])+'.pkl')
        region_df = pytrend.interest_by_region(resolution='COUNTRY', inc_low_vol=False, inc_geo_code=False)
        region_df.to_pickle('Data/'+str(kw_list[0])+'_regions'+'.pkl')
        print('term with regional data saved')
        return(search_df, region_df)
    else:
        print('term has been found..')
        pytrend = TrendReq(hl='en-US', tz=360)   
        term_df = pd.read_pickle('Data/'+str(kw_list[0])+'.pkl')
        region_df = pd.read_pickle('Data/'+str(kw_list[0])+'_regions'+'.pkl')
        print(term_df.head())
        return(term_df, region_df)
        
        
def plot_interest_in_time(df2, kw_list):

    from bokeh.io import show, output_file
    from bokeh.plotting import figure
    from bokeh.io import output_notebook
    from datetime import datetime as dt
    from bokeh.models import DatetimeTickFormatter

    output_notebook()
    TOOLS = 'save,pan,box_zoom,reset,wheel_zoom,hover'
    TOOLTIPS = [
        ('month', '@x{datetime}'),
        ('Total searches', '@y'),
    ]

    p = figure(title="Monthly-wise total search of term - "+str(kw_list[0]), y_axis_type="linear", plot_height = 400,
               tools = TOOLS, tooltips = TOOLTIPS, plot_width = 800)


    p.xaxis.axis_label = 'Month'
    p.yaxis.axis_label = 'Total search'
    #p.circle(2010, temp_df.IncidntNum.min(), size = 10, color = 'red')

    p.line(df2.index, df2[str(kw_list[0])],line_color="purple", line_width = 3)

    p.xaxis.formatter=DatetimeTickFormatter(

            days=["%d %B %Y"],
            months=["%d %B %Y"],
            years=["%d %B %Y"],
        )
    # output_file("line_chart.html", title="Line Chart")
    show(p)


def interest_test(kw_list, pytrend):
    #create patload
    pytrend.build_payload(kw_list, timeframe='all')
    interest_over_time_df = pytrend.interest_over_time()
    search_df.to_pickle('Data/trends/'+str(kw_list[0])+'.pkl')
    return(interest_over_time_df)
    #print(interest_over_time_df)
#edef

In [ ]:
kw_list = ['greece crisis']

#time_df, regions_df = get_interest_and_region(kw_list, pytrend)

df = interest_test(kw_list,pytrend)
plot_interest_in_time(df, kw_list)


In [ ]:
## Downsampling to monthly
df2 = time_df.resample('M', on='date').mean()
print(df2.index)

In [ ]:
plot_interest_in_time(df2,kw_list)

In [ ]:
# #add year and month
# term_csv['Year'] = term_csv.date.apply(lambda x: x.year)
# term_csv['Month'] = term_csv.date.apply(lambda x: x.month)
# term_csv.head()

In [ ]:
# temp_df = term_csv.groupby(['Year']).mean().reset_index()

# temp_df

In [ ]:
region_df = pd.read_pickle('Data/crisis_regions'+'.pkl')

In [ ]:
region_df.index[0]

## Google trends (Interest over time)

In [1]:
import pandas as pd
from pytrends.request import TrendReq
import os  
import numpy as np
pytrend = TrendReq(hl='en-US', tz=360)

In [15]:
def interest_test(kw_list, pytrend):
    #create patload
    pytrend.build_payload(kw_list, timeframe='all')
    interest_over_time_df = pytrend.interest_over_time()
    interest_over_time_df.to_pickle('Data/trends/'+str(kw_list[0].replace(" ", "_"))+'.pkl')
    region_df = pytrend.interest_by_region(resolution='COUNTRY', inc_low_vol=False, inc_geo_code=False)
    region_df.to_pickle('Data/trends/'+str(kw_list[0].replace(" ", "_"))+'_regions'+'.pkl')
    #return(interest_over_time_df)
    #print(interest_over_time_df)
#edef
def plot_interest_in_time(df2, kw_list):

    from bokeh.io import show, output_file
    from bokeh.plotting import figure
    from bokeh.io import output_notebook
    from datetime import datetime as dt
    from bokeh.models import DatetimeTickFormatter

    output_notebook()
    TOOLS = 'save,pan,box_zoom,reset,wheel_zoom,hover'
    TOOLTIPS = [
        ('month', '@x{datetime}'),
        ('Total searches', '@y'),
    ]

    p = figure(title="Monthly-wise total search of term - "+str(kw_list[0]), y_axis_type="linear", plot_height = 400,
               tools = TOOLS, tooltips = TOOLTIPS, plot_width = 800)


    p.xaxis.axis_label = 'Month'
    p.yaxis.axis_label = 'Total search'
    #p.circle(2010, temp_df.IncidntNum.min(), size = 10, color = 'red')

    p.line(df2.index, df2[str(kw_list[0])],line_color="purple", line_width = 3)

    p.xaxis.formatter=DatetimeTickFormatter(

            days=["%d %B %Y"],
            months=["%d %B %Y"],
            years=["%d %B %Y"],
        )
    # output_file("line_chart.html", title="Line Chart")
    show(p)
#edef

In [7]:
tags = [ 'humanitarian crisis',
         'natural disaster',
         'environmental crisis',
         'disability',
         'gender',
         'genital mutilation',
         'racism',
         'genocide',
         'civil war',
         'terrorism',
         'infectious disease',
         'political revolution',
         'political prisoner',
         'amnesty',
         'corruption',
         'health awareness',
         'gender inequality',
         'rape',
         'ebola',
         'aids']

In [ ]:
# Download country and terms
region_df = pd.read_pickle('Data/crisis_regions'+'.pkl')
for country in region_df.index[:10]:
    for term in tags:
        kw_list = [country + ' '+ term]
        interest_test(kw_list, pytrend)

In [22]:
# plot an example
ex_df = pd.read_pickle('Data/trends/Algeria_genocide.pkl')
print(ex_df)
kw_li = ['Algeria genocide']
plot_interest_in_time(ex_df, kw_li)


            Algeria genocide isPartial
date                                  
2004-01-01                 0     False
2004-02-01                 0     False
2004-03-01                 0     False
2004-04-01               100     False
2004-05-01                 0     False
2004-06-01                 0     False
2004-07-01                 0     False
2004-08-01                 0     False
2004-09-01                77     False
2004-10-01                78     False
2004-11-01                 0     False
2004-12-01                 0     False
2005-01-01                 0     False
2005-02-01                69     False
2005-03-01                 0     False
2005-04-01                 0     False
2005-05-01                 0     False
2005-06-01                 0     False
2005-07-01                57     False
2005-08-01                 0     False
2005-09-01                 0     False
2005-10-01                 0     False
2005-11-01                 0     False
2005-12-01               

Loading BokehJS ...